In [28]:
# tf2.0版本
import tensorflow.compat.v1 as tf
tf.disable_eager_execution()         #可以用于从TensorFlow 1.x到2.x的复杂迁移项目的程序开头
    
hello = tf.constant('hello,tf')
sess = tf.Session()
print(sess.run(hello))

b'hello,tf'


In [29]:
features = {
    'sale': [1.2, 2.3, 1.2, 1.5, 2.2]
}

#特征列
sale = tf.feature_column.numeric_column("sale", default_value=0.0, normalizer_fn=tf.log)

In [30]:
sale

NumericColumn(key='sale', shape=(1,), default_value=(0.0,), dtype=tf.float32, normalizer_fn=<function log at 0x00000000081264C0>)

In [5]:
tf.log([1.0])

<tf.Tensor 'Log_1:0' shape=(1,) dtype=float32>

In [6]:
#组合特征列
columns = [
    sale
]

#输入层（数据，特征列）
inputs = tf.feature_column.input_layer(features, columns)

#初始化并运行
init = tf.global_variables_initializer()
sess.run(tf.tables_initializer())
sess.run(init)

v=sess.run(inputs)
print(v)

Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
Instructions for updating:
The old _FeatureColumn APIs are being deprecated. Please use the new FeatureColumn APIs instead.
[[0.1823216]
 [0.8329091]
 [0.1823216]
 [0.4054651]
 [0.7884574]]


In [11]:
from tensorflow.python.feature_column.feature_column import _LazyBuilder
from tensorflow.compat.v1 import feature_column

In [33]:
def test_categorical_column_with_vocabulary_list():
    color_column = tf.feature_column.categorical_column_with_vocabulary_list(
        'color', ['R', 'G', 'B', 'X', 'Y', 'Z']
    )
    color_column = tf.feature_column.indicator_column(color_column)
    print(color_column)
    features = {'color':['R', 'G', 'B', 'X', 'Y', 'Z'],}

    inputs = tf.feature_column.input_layer(features, color_column)
    
    #初始化并运行
    init = tf.global_variables_initializer()
    sess.run(tf.tables_initializer())
    sess.run(init)

    v=sess.run(inputs)
    print(v)

test_categorical_column_with_vocabulary_list()

IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='color', vocabulary_list=('R', 'G', 'B', 'X', 'Y', 'Z'), dtype=tf.string, default_value=-1, num_oov_buckets=0))


ValueError: Fetch argument 'color' cannot be interpreted as a Tensor. ("The name 'color' refers to an Operation not in the graph.")

In [13]:

 
#特征数据
features = {
    'sex': ['male', 'male', 'female', 'female'],
}
#特征列
sex_column = tf.feature_column.categorical_column_with_vocabulary_list('sex', ['male', 'female'])
sex_column = tf.feature_column.indicator_column(sex_column)
#组合特征列
columns = [sex_column]
#输入层（数据，特征列）
inputs = tf.feature_column.input_layer(features, columns)
#初始化并运行
init = tf.global_variables_initializer()
sess.run(tf.tables_initializer())
sess.run(init)
 
v=sess.run(inputs)
print(v)

[[1. 0.]
 [1. 0.]
 [0. 1.]
 [0. 1.]]


In [24]:
def test_crossed_column():
    """ crossed column测试 """
    featrues = { #4个 样本
        'price': [['A'], ['A'], ['B'], ['B']],
        'color': [['R'], ['R'], ['G'], ['R']]
    }
    price = feature_column.categorical_column_with_vocabulary_list('price', ['A', 'B', 'C'])
    color = feature_column.categorical_column_with_vocabulary_list('color', ['R', 'G'])
    p_x_c = feature_column.crossed_column([price, color], 6) #6有点小，可能冲突。8是冲突的
    
    p_x_c_identy = feature_column.indicator_column(p_x_c)
    p_x_c_identy_dense_tensor = feature_column.input_layer(featrues, [p_x_c_identy])
    
    sess=tf.Session()
    #初始化并运行
    init = tf.global_variables_initializer()
    sess.run(tf.tables_initializer())
    sess.run(init)

    v=sess.run(p_x_c_identy_dense_tensor)
    print(v)

test_crossed_column()

[[0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 1.]]


In [27]:
#特征数据
features = {   #5个样本
    'sex': [1, 1, 0, 1, 1],
    'department': ['sport', 'sport', 'drawing', 'gardening', 'travelling'],
}

#特征列
department =  feature_column.categorical_column_with_vocabulary_list('department', ['sport','drawing','gardening','travelling'], dtype=tf.string)
sex =  feature_column.categorical_column_with_identity('sex', num_buckets=2, default_value=0)

sex_department =  feature_column.crossed_column([department,sex], 25) #交叉特征

sex_department_emb =  feature_column.embedding_column(sex_department, 5, combiner='sqrtn')

sex_department_indicator = feature_column.indicator_column(sex_department)


#组合特征列
columns = [
    sex_department_emb
]

columns2 = [
    sex_department_indicator
]


#输入层（数据，特征列）
sex_department_emb_tensor = feature_column.input_layer(features, columns)

sex_department_indicator_tensor = feature_column.input_layer(features, columns2)


#初始化并运行
init = tf.global_variables_initializer()
sess.run(tf.tables_initializer())
sess.run(init)
print("emb----------")
v=sess.run(sex_department_emb_tensor)
print(v)

print("indicator ----------")
v=sess.run(sex_department_indicator_tensor)
print(v)



emb----------
[[ 4.64572102e-01  4.69987363e-01 -1.02255970e-01 -1.37822866e-01
  -7.50116289e-01]
 [ 4.64572102e-01  4.69987363e-01 -1.02255970e-01 -1.37822866e-01
  -7.50116289e-01]
 [ 1.64502218e-01  3.99217457e-01  4.53614688e-04 -3.34363818e-01
   8.38656902e-01]
 [-8.04516315e-01  3.04193556e-01  5.24926543e-01  3.79642874e-01
  -1.08995214e-01]
 [-6.81539595e-01 -2.11162984e-01  5.88389575e-01 -1.94979578e-01
  -1.68603897e-01]]
indicator ----------
[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.
  0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.
  0.]]
